In [1]:
import os
import openai
import json
import pandas as pd
from datasets import load_dataset

from google import genai

client = genai.Client(api_key="")

sib_languages = ['ace_Arab', 'ace_Latn', 'acm_Arab', 'acq_Arab', 'aeb_Arab', 'afr_Latn', 'ajp_Arab', 'aka_Latn', 'als_Latn', 'amh_Ethi', 'apc_Arab', 'arb_Arab', 'arb_Latn', 'ars_Arab', 'ary_Arab', 'arz_Arab', 'asm_Beng', 'ast_Latn', 'awa_Deva', 'ayr_Latn', 'azb_Arab', 'azj_Latn', 'bak_Cyrl', 'bam_Latn', 'ban_Latn', 'bel_Cyrl', 'bem_Latn', 'ben_Beng', 'bho_Deva', 'bjn_Arab', 'bjn_Latn', 'bod_Tibt', 'bos_Latn', 'bug_Latn', 'bul_Cyrl', 'cat_Latn', 'ceb_Latn', 'ces_Latn', 'cjk_Latn', 'ckb_Arab', 'crh_Latn', 'cym_Latn', 'dan_Latn', 'deu_Latn', 'dik_Latn', 'dyu_Latn', 'dzo_Tibt', 'ell_Grek', 'eng_Latn', 'epo_Latn', 'est_Latn', 'eus_Latn', 'ewe_Latn', 'fao_Latn', 'fij_Latn', 'fin_Latn', 'fon_Latn', 'fra_Latn', 'fur_Latn', 'fuv_Latn', 'gaz_Latn', 'gla_Latn', 'gle_Latn', 'glg_Latn', 'grn_Latn', 'guj_Gujr', 'hat_Latn', 'hau_Latn', 'heb_Hebr', 'hin_Deva', 'hne_Deva', 'hrv_Latn', 'hun_Latn', 'hye_Armn', 'ibo_Latn', 'ilo_Latn', 'ind_Latn', 'isl_Latn', 'ita_Latn', 'jav_Latn', 'jpn_Jpan', 'kab_Latn', 'kac_Latn', 'kam_Latn', 'kan_Knda', 'kas_Arab', 'kas_Deva', 'kat_Geor', 'kaz_Cyrl', 'kbp_Latn', 'kea_Latn', 'khk_Cyrl', 'khm_Khmr', 'kik_Latn', 'kin_Latn', 'kir_Cyrl', 'kmb_Latn', 'kmr_Latn', 'knc_Arab', 'knc_Latn', 'kon_Latn', 'kor_Hang', 'lao_Laoo', 'lij_Latn', 'lim_Latn', 'lin_Latn', 'lit_Latn', 'lmo_Latn', 'ltg_Latn', 'ltz_Latn', 'lua_Latn', 'lug_Latn', 'luo_Latn', 'lus_Latn', 'lvs_Latn', 'mag_Deva', 'mai_Deva', 'mal_Mlym', 'mar_Deva', 'min_Arab', 'min_Latn', 'mkd_Cyrl', 'mlt_Latn', 'mni_Beng', 'mos_Latn', 'mri_Latn', 'mya_Mymr', 'nld_Latn', 'nno_Latn', 'nob_Latn', 'npi_Deva', 'nqo_Nkoo', 'nso_Latn', 'nus_Latn', 'nya_Latn', 'oci_Latn', 'ory_Orya', 'pag_Latn', 'pan_Guru', 'pap_Latn', 'pbt_Arab', 'pes_Arab', 'plt_Latn', 'pol_Latn', 'por_Latn', 'prs_Arab', 'quy_Latn', 'ron_Latn', 'run_Latn', 'rus_Cyrl', 'sag_Latn', 'san_Deva', 'sat_Olck', 'scn_Latn', 'shn_Mymr', 'sin_Sinh', 'slk_Latn', 'slv_Latn', 'smo_Latn', 'sna_Latn', 'snd_Arab', 'som_Latn', 'sot_Latn', 'spa_Latn', 'srd_Latn', 'srp_Cyrl', 'ssw_Latn', 'sun_Latn', 'swe_Latn', 'swh_Latn', 'szl_Latn', 'tam_Taml', 'taq_Latn', 'taq_Tfng', 'tat_Cyrl', 'tel_Telu', 'tgk_Cyrl', 'tgl_Latn', 'tha_Thai', 'tir_Ethi', 'tpi_Latn', 'tsn_Latn', 'tso_Latn', 'tuk_Latn', 'tum_Latn', 'tur_Latn', 'twi_Latn', 'tzm_Tfng', 'uig_Arab', 'ukr_Cyrl', 'umb_Latn', 'urd_Arab', 'uzn_Latn', 'vec_Latn', 'vie_Latn', 'war_Latn', 'wol_Latn', 'xho_Latn', 'ydd_Hebr', 'yor_Latn', 'yue_Hant', 'zho_Hans', 'zho_Hant', 'zsm_Latn', 'zul_Latn']



In [2]:
prompt_sib="You are an assistant able to classify topics in texts. \n\nGiven the categories travel, politics, science, sports, technology, health, nature, entertainment, geography, business, disasters, crime, education, religion; what is the topic of the {} statement below? Return only the category. \n\ntext: {} \ncategory:\n\n"

In [3]:
import re
import time

categories = ["travel", "politics", "science", "sports", "technology", "health", "nature", "entertainment", "geography",
"business", "disasters", "crime", "education", "religion"]

def get_category(prompt, max_retries=3, backoff_factor=2):
    for attempt in range(max_retries):
        try:
            response = client.models.generate_content(
                        model="gemini-2.0-flash",
                        contents=prompt,
                    )
            reply = response.text.lower()
            # print(reply)
            category = ''
            if 'travel' in reply:
                category = categories[0]
            elif 'politics' in reply:
                category = categories[1]
            elif 'science' in reply:
                category = categories[2]
            elif 'sports' in reply:
                category = categories[3]
            elif 'technology' in reply:
                category = categories[4]
            elif 'health' in reply:
                category = categories[5]
            elif 'nature' in reply:
                category = categories[6]
            elif 'entertainment' in reply:
                category = categories[7]
            elif 'geography' in reply:
                category = categories[8]
            elif 'business' in reply:
                category = categories[9]
            elif 'disasters' in reply:
                category = categories[10]
            elif 'crime' in reply:
                category = categories[11]
            elif 'education' in reply:
                category = categories[12]
            elif 'religion' in reply:
                category = categories[13]
            else:
                category = 'invalid'
            return reply, category
        except Exception as e:
            print(f"Attempt {attempt + 1} failed with error: {e}")
            if attempt < max_retries - 1:
                sleep_time = backoff_factor ** attempt
                print(f"Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
            else:
                print("Max retries reached. Returning failure label.")
                return "", 'invalid'


In [4]:
df = pd.read_csv('sib_gpt4o_results_march2025.csv')
code_to_name_map = df.set_index('language code')['Language name'].to_dict()

In [5]:
languages_to_run = sib_languages[0:69]

In [6]:
from tqdm import tqdm
result_categories = {}
result_accuracies = {}
gpt_replies = {}
result_inaccuracies = {}

for language_code in languages_to_run:
    completed = []
    if language_code in completed:
        continue
    print(language_code)
    accurate = 0
    errors = 0
    language_name = code_to_name_map[language_code]
    
    result_categories[language_code]=[]
    gpt_replies[language_code] = []
    dataset = load_dataset("Davlan/sib200_14classes", language_code) 

    size = len(dataset['test'])
    for i in tqdm(range(size)): #length of devtest
        text = dataset['test'][i]['text']
        prompt = prompt_sib.format(language_name, text)
        reply, category = get_category(prompt)
        gpt_replies[language_code].append(reply)
        result_categories[language_code].append(category)
        if category == dataset['test'][i]['category']:
            accurate+=1
        if category == 'invalid':
            errors+=1
    result_accuracies[language_code] = accurate
    result_inaccuracies[language_code] = errors

    df = pd.DataFrame({
    "text": dataset['test']['text'],
    "gemini_label": result_categories[language_code]
    })

    # Save to CSV
    df.to_csv("sib14_results_prompt2_gemini2/{}.csv".format(language_code), index=False)

    df2 = pd.DataFrame({
    "text": dataset['test']['text'],
    "gemini_reply": gpt_replies[language_code]
    })

    # Save to CSV
    df2.to_csv("sib14_prompt2_gemini2_replies/{}.csv".format(language_code), index=False)

    print(accurate)

ace_Arab


 24%|██████████████████████████████████████▋                                                                                                                             | 289/1225 [01:57<06:02,  2.58it/s]

Attempt 1 failed with error: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
Retrying in 1 seconds...


 62%|████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                               | 754/1225 [04:58<03:00,  2.61it/s]

Attempt 1 failed with error: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
Retrying in 1 seconds...


...

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1225/1225 [08:10<00:00,  2.50it/s]

898


In [7]:
print(result_accuracies)

{'ace_Arab': 727, 'ace_Latn': 848, 'acm_Arab': 893, 'acq_Arab': 915, 'aeb_Arab': 917, 'afr_Latn': 921, 'ajp_Arab': 914, 'aka_Latn': 875, 'als_Latn': 907, 'amh_Ethi': 914, 'apc_Arab': 927, 'arb_Arab': 927, 'arb_Latn': 925, 'ars_Arab': 915, 'ary_Arab': 914, 'arz_Arab': 910, 'asm_Beng': 909, 'ast_Latn': 897, 'awa_Deva': 912, 'ayr_Latn': 728, 'azb_Arab': 847, 'azj_Latn': 906, 'bak_Cyrl': 899, 'bam_Latn': 727, 'ban_Latn': 890, 'bel_Cyrl': 902, 'bem_Latn': 841, 'ben_Beng': 906, 'bho_Deva': 897, 'bjn_Arab': 809, 'bjn_Latn': 891, 'bod_Tibt': 889, 'bos_Latn': 915, 'bug_Latn': 811, 'bul_Cyrl': 918, 'cat_Latn': 908, 'ceb_Latn': 914, 'ces_Latn': 916, 'cjk_Latn': 529, 'ckb_Arab': 923, 'crh_Latn': 837, 'cym_Latn': 896, 'dan_Latn': 911, 'deu_Latn': 921, 'dik_Latn': 526, 'dyu_Latn': 638, 'dzo_Tibt': 859, 'ell_Grek': 909, 'eng_Latn': 915, 'epo_Latn': 911, 'est_Latn': 919, 'eus_Latn': 908, 'ewe_Latn': 827, 'fao_Latn': 906, 'fij_Latn': 879, 'fin_Latn': 920, 'fon_Latn': 716, 'fra_Latn': 912, 'fur_Latn': 9

In [11]:
ra2={'hin_Deva': 907, 'hne_Deva': 899, 'hrv_Latn': 925, 'hun_Latn': 911, 'hye_Armn': 897, 'ibo_Latn': 891, 'ilo_Latn': 906, 'ind_Latn': 913, 'isl_Latn': 898, 'ita_Latn': 916, 'jav_Latn': 926, 'jpn_Jpan': 914, 'kab_Latn': 758, 'kac_Latn': 780, 'kam_Latn': 734, 'kan_Knda': 901, 'kas_Arab': 882, 'kas_Deva': 839, 'kat_Geor': 910, 'kaz_Cyrl': 904, 'kbp_Latn': 720, 'kea_Latn': 907, 'khk_Cyrl': 903, 'khm_Khmr': 916, 'kik_Latn': 806, 'kin_Latn': 916, 'kir_Cyrl': 899, 'kmb_Latn': 631, 'kmr_Latn': 868, 'knc_Arab': 275, 'knc_Latn': 480, 'kon_Latn': 842, 'kor_Hang': 909, 'lao_Laoo': 918, 'lij_Latn': 898, 'lim_Latn': 890, 'lin_Latn': 882, 'lit_Latn': 904, 'lmo_Latn': 904, 'ltg_Latn': 911, 'ltz_Latn': 924, 'lua_Latn': 707, 'lug_Latn': 851, 'luo_Latn': 747, 'lus_Latn': 898, 'lvs_Latn': 906, 'mag_Deva': 909, 'mai_Deva': 911, 'mal_Mlym': 913, 'mar_Deva': 901, 'min_Arab': 796, 'min_Latn': 906, 'mkd_Cyrl': 922, 'mlt_Latn': 912, 'mni_Beng': 882, 'mos_Latn': 485, 'mri_Latn': 869, 'mya_Mymr': 905, 'nld_Latn': 909, 'nno_Latn': 905, 'nob_Latn': 917, 'npi_Deva': 894, 'nqo_Nkoo': 674, 'nso_Latn': 872, 'nus_Latn': 305, 'nya_Latn': 884, 'oci_Latn': 920, 'ory_Orya': 914, 'pag_Latn': 871}

In [12]:
ra3={'pan_Guru': 919, 'pap_Latn': 918, 'pbt_Arab': 888, 'pes_Arab': 904, 'plt_Latn': 872, 'pol_Latn': 908, 'por_Latn': 897, 'prs_Arab': 914, 'quy_Latn': 763, 'ron_Latn': 907, 'run_Latn': 875, 'rus_Cyrl': 915, 'sag_Latn': 768, 'san_Deva': 852, 'sat_Olck': 887, 'scn_Latn': 908, 'shn_Mymr': 883, 'sin_Sinh': 904, 'slk_Latn': 907, 'slv_Latn': 914, 'smo_Latn': 911, 'sna_Latn': 897, 'snd_Arab': 905, 'som_Latn': 882, 'sot_Latn': 891, 'spa_Latn': 905, 'srd_Latn': 912, 'srp_Cyrl': 920, 'ssw_Latn': 866, 'sun_Latn': 911, 'swe_Latn': 910, 'swh_Latn': 926, 'szl_Latn': 874, 'tam_Taml': 903, 'taq_Latn': 572, 'taq_Tfng': 385, 'tat_Cyrl': 891, 'tel_Telu': 901, 'tgk_Cyrl': 920, 'tgl_Latn': 914, 'tha_Thai': 909, 'tir_Ethi': 889, 'tpi_Latn': 911, 'tsn_Latn': 891, 'tso_Latn': 889, 'tuk_Latn': 898, 'tum_Latn': 868, 'tur_Latn': 902, 'twi_Latn': 879, 'tzm_Tfng': 731, 'uig_Arab': 852, 'ukr_Cyrl': 908, 'umb_Latn': 627, 'urd_Arab': 906, 'uzn_Latn': 898, 'vec_Latn': 903, 'vie_Latn': 907, 'war_Latn': 911, 'wol_Latn': 840, 'xho_Latn': 905, 'ydd_Hebr': 907, 'yor_Latn': 868, 'yue_Hant': 912, 'zho_Hans': 919, 'zho_Hant': 921, 'zsm_Latn': 922, 'zul_Latn': 916}

In [13]:
ra = result_accuracies | ra2| ra3

In [16]:
import json

# Save to a JSON file
with open("result_accuracies_sib14_gemini2.json", "w", encoding="utf-8") as f:
    json.dump(ra, f, ensure_ascii=False, indent=4)

df = pd.DataFrame({
    "Language name": [code_to_name_map[language_code] for language_code in sib_languages],
    "Language code": sib_languages,
    "Accuracy": [round(ra[language_code]*100/1225, 1) for language_code in sib_languages]
})

# Save to CSV
df.to_csv("sib14_gemini2_results_prompt2.csv", index=False)